<a href="https://colab.research.google.com/github/tldtldcpfl/JoChaeRin/blob/master/bio_textmining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Goal

-환자기록을 기반으로 정형 데이터(예.표준화 질병코드)와 비정형 텍스트 마이닝에 nlp 기술을 이용한다.이를 통해 환자 집단 계층화와 질병과의 상관관계 분석을 한다. 예를 들어, 약물 남용 코드를 가진 환자가 행동장애 질병간의 동시질환 가능성 예상에 도움이 될 수 있음을 보인다. 

-30만개 이상의 의료개념 및 관계에 대한 계층화된 시맨틱 네트워크 개발

-웹크롤링을 이용한 BioNELL 무한 언어학습기 연구 


In [ ]:
# 1.(동시 출현 빈도 계산) 두 개체간의 관계성 확인하기 위해 한 문장 내에 존재하는 개체들간의 동시 출현 빈도(예.100회 이상 출현) 기반의 관계를 확인
# 2. 개체를 연결하는 동사의 긍정/부정을 통한 두 개체간의 관계성 파악 (pos/neg)

# 1. 개체간의 동시 출현 연결성을 구조적으로 파악하기 위해 시각화한다. 
#   - 개체들을 네트워크의 노드들로 표현   



In [ ]:
!pip install -U tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 114kB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 450kB 26.8MB/s 
     |████████████████████████████████| 3.8MB 31.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=f324d6e86632eccc4ff0b0eb558886ad169744ba98e681773772c30aeb702973
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from xgboost import DMatrix

In [ ]:
print('Tensorflow ', tf.__version__)

Tensorflow  2.0.0


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

In [ ]:
train_data = pd.read_json('/content/train.json', lines=True)


In [ ]:
test_data = pd.read_json('/content/test.json', lines=True)

In [ ]:
submit = pd.read_csv('/content/sample_submission.csv')

In [ ]:
submit.head()

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.0,0.0,0.0,0.0,0.0
1,id_00073f8be_1,0.0,0.0,0.0,0.0,0.0
2,id_00073f8be_2,0.0,0.0,0.0,0.0,0.0
3,id_00073f8be_3,0.0,0.0,0.0,0.0,0.0
4,id_00073f8be_4,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_data.head()

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,..."


In [ ]:
print(train_data.shape)


(2400, 19)


In [ ]:
print(train_data.columns)

Index(['index', 'id', 'sequence', 'structure', 'predicted_loop_type',
       'signal_to_noise', 'SN_filter', 'seq_length', 'seq_scored',
       'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10',
       'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C'],
      dtype='object')


Feature Extraction

In [ ]:
def in_out_shape(data, input_shape=107, output_shape=68, contra=True):
    if type(data) != 'list':
        d = list(data)
    else:
        d = data
        
    if (input_shape == 107 and output_shape == 107 and contra == True)\
        or (input_shape == 68 and output_shape == 107):
        
        d = d[:68]
        d.extend(d[:39])
        return d
    
    elif (input_shape == 107 and output_shape == 68)\
        or (input_shape == 68 and output_shape == 68):
        
        d = d[:68]
        return d
    
    elif input_shape == 107 and output_shape == 107 and contra == False:
        d = d[:107]
        return d
    elif input_shape == 130 and output_shape == 130 and contra == False:
        d = d[:130]
        return d

In [ ]:
def f1_ls_to_np(data, ishp, oshp, c):
    d = in_out_shape(data, ishp, oshp, c)
    ls = []
    for i in range(len(d)):
        if d[i] == 'A':
            ls.append([1, 0, 0, 0])
        elif d[i] == 'U':
            ls.append([0, 1, 0, 0])
        elif d[i] == 'G':
            ls.append([0, 0, 1, 0])
        else:
            ls.append([0, 0, 0, 1])
    ls = np.array(ls)
    return ls

def bases(data, i_shp = 107, o_shp = 107, c = True):
    obj = np.expand_dims(f1_ls_to_np(data[0], i_shp, o_shp, c), axis=0)
    for i in range(1, len(data)):
        obj = np.concatenate((obj, np.expand_dims(f1_ls_to_np(data[i], i_shp, o_shp, c), axis=0)))
    return obj

In [ ]:
def struct_1_hot(data, index, isp, test):
    if test == True and isp == 107:
        bases = ts_df_107.sequence[index]
    elif test == True and isp == 130:
        bases = ts_df_130.sequence[index]
    else:
        bases = train_data.sequence[index]
    idxs = []
    struct_1_hot = [i for i in range(0, len(data))]
    for idx, item in enumerate(data):
        if item == '.':
            struct_1_hot[idx] = [0, 0, 0, 0, 0, 0]
        elif item == '(':
            idxs.append(idx)
        elif item == ')':
            first = idxs.pop()
            second = idx
            if bases[first] == 'U':
                if bases[second] == 'G': # U-G
                    struct_1_hot[first] = [1, 0, 0, 0, 0, 0]
                    struct_1_hot[second] = [1, 0, 0, 0, 0, 0]
                else: # U-A
                    struct_1_hot[first] = [0, 0, 0, 1, 0, 0]
                    struct_1_hot[second] = [0, 0, 0, 1, 0, 0]
            elif bases[first] == 'G':
                if bases[second] == 'U': # G-U
                    struct_1_hot[first] = [0, 1, 0, 0, 0, 0]
                    struct_1_hot[second] = [0, 1, 0, 0, 0, 0]
                else: # G-C
                    struct_1_hot[first] = [0, 0, 0, 0, 1, 0]
                    struct_1_hot[second] = [0, 0, 0, 0, 1, 0]
            elif bases[first] == 'A': # A-U
                struct_1_hot[first] = [0, 0, 1, 0, 0, 0]
                struct_1_hot[second] = [0, 0, 1, 0, 0, 0]
            else: #C-G
                struct_1_hot[first] = [0, 0, 0, 0, 0, 1]
                struct_1_hot[second] = [0, 0, 0, 0, 0, 1]
    return struct_1_hot

def f2_ls_to_np(data, index, isp, osp, c, t):
    d = list(data)
    d = struct_1_hot(d, index, isp, t)
    d = in_out_shape(d, isp, osp, c)
    d = np.array(d)
    return d

def structs(data, isp = 107, osp = 107, c = True, t = False):
    obj = np.expand_dims(f2_ls_to_np(data[0], 0, isp, osp, c, t), axis=0)
    for i in range(1, len(data)):
        obj = np.concatenate((obj, np.expand_dims(f2_ls_to_np(data[i], i, isp, osp, c, t), axis=0)))
    return obj

In [ ]:
def f3_ls_to_np(data, isp, osp, c):
    d = in_out_shape(data, isp, osp, c)
    ls = []
    for i in range(len(d)):
        if d[i] == 'E':#Dangling End
            ls.append([1, 0, 0, 0, 0, 0])
        elif d[i] == 'S':#Paired Stem
            ls.append([0, 1, 0, 0, 0, 0])
        elif d[i] == 'H':#Hairpin
            ls.append([0, 0, 1, 0, 0, 0])
        elif d[i] == 'B':#Bulge
            ls.append([0, 0, 0, 1, 0, 0])
        elif d[i] == 'x':#eXternal Loop
            ls.append([0, 0, 0, 0, 1, 0])
        else:# 'I' #Internal Loop
            ls.append([0, 0, 0, 0, 0, 1])
    ls = np.array(ls)
    return ls

def loop_types(data, isp = 107, osp = 107, c = True):
    obj = np.expand_dims(f3_ls_to_np(data[0], isp, osp, c), axis=0)
    for i in range(1, len(data)):
        obj = np.concatenate((obj, np.expand_dims(f3_ls_to_np(data[i], isp, osp, c), axis=0)))
    return obj

Feature-1 // bases

In [ ]:
f1 = train_data.sequence

In [ ]:
bases_68_107 = bases(f1, 107, 107, True)

bases_68 = bases(f1, 68, 68, False)

bases_107 = bases(f1, 107, 107, False)
print(bases_107.shape)

(2400, 107, 4)


In [ ]:
print(bases_107)

[[[0 0 1 0]
  [0 0 1 0]
  [1 0 0 0]
  ...
  [1 0 0 0]
  [1 0 0 0]
  [0 0 0 1]]

 [[0 0 1 0]
  [0 0 1 0]
  [1 0 0 0]
  ...
  [1 0 0 0]
  [1 0 0 0]
  [0 0 0 1]]

 [[0 0 1 0]
  [0 0 1 0]
  [1 0 0 0]
  ...
  [1 0 0 0]
  [1 0 0 0]
  [0 0 0 1]]

 ...

 [[0 0 1 0]
  [0 0 1 0]
  [1 0 0 0]
  ...
  [1 0 0 0]
  [1 0 0 0]
  [0 0 0 1]]

 [[0 0 1 0]
  [0 0 1 0]
  [1 0 0 0]
  ...
  [1 0 0 0]
  [1 0 0 0]
  [0 0 0 1]]

 [[0 0 1 0]
  [0 0 1 0]
  [1 0 0 0]
  ...
  [1 0 0 0]
  [1 0 0 0]
  [0 0 0 1]]]


Feature-2 // structures

In [ ]:
f2 = train_data.structure

structs_68_107 = structs(f2, 107, 107, True)

structs_68 = structs(f2, 68, 68, False)

structs_107 = structs(f2, 107, 107, False)
print(structs_107.shape)

(2400, 107, 6)


Feature-3 // Loop type

In [ ]:
f3 = train_data.predicted_loop_type

loop_types_68_107 = loop_types(f3, 107, 107, True)

loop_types_68 = loop_types(f3, 68, 68, False)

loop_types_107 = loop_types(f3, 107, 107, False)
print(loop_types_107.shape)

(2400, 107, 6)


Target Extraction

In [ ]:
def t_ls_to_np(data, isp, osp, c):
    d = in_out_shape(data, isp, osp, c)
    d = np.array(d)
    d = np.expand_dims(d, axis=0)
    return d

def target(data, isp = 68, osp = 107, c = True):
    obj = t_ls_to_np(data[0], isp, osp, c)
    for i in range(1, len(data)):
        obj = np.concatenate((obj, t_ls_to_np(data[i], isp, osp, c)), axis=0)
    return ob

Target-1 // reactivity

In [ ]:
def filtrate(t, e):
    t = np.reshape(t, (163200, 1))
    e = np.reshape(e, (163200, 1))
    
    e_fltr = np.squeeze(e <= 0.3)
    
    t = t[e_fltr]
    
    t_mean = np.mean(t)
    t_std = np.std(t)
    
    t_fltr = np.abs(t - t_mean) < 3 * t_std
    t_fltr = np.squeeze(t_fltr)
    
    t = t[t_fltr]
    
    return features[e_fltr][t_fltr], t